In [1]:
using Paths
using Plots
using DataFrames
using CSV

In [2]:
CSV.open("../data/randomFixed|100x100|maxCost=2.0.csv")

IOStream(<file ../data/randomFixed|100x100|maxCost=2.0.csv>)

In [ ]:
X=100
Y=100
maxCost = 2.0
numWalkers = 10
numLocations = 10
scenario = Paths.RANDOM_FIXED
logisticPatchImprovement=false

PIS = [0.04] #[0.01, 0.02, 0.04, 0.08] 
PRS = [0.0004] # [0.001, 0.002, 0.004, 0.008]

filetext = "data/0.04_$(scenario.description)|$(X)x$(Y)|maxCost=$(maxCost).csv"
@info "saving to $(filetext)"

F = 200
upf = 100
df = DataFrame()

for pI ∈ PIS
    for pR ∈ PRS
        sim::Simulation = Paths.MakeSimulation(
            X=X, Y=Y, numWalkers=numWalkers, numLocations=numLocations,
            patchImprovement=pI, patchRecovery=pR,
            scenario=scenario, maxCost=maxCost,
            logisticPatchImprovement=logisticPatchImprovement)

        averageTravelCosts = [averageTravelCost(sim)]
        totalImprovements = [totalImprovement(sim)]
        steps = [0]

        for f ∈ 1:F
            for u ∈ 1:upf
                print("\r$(pI) $(pR) $(f)/$(F) $(u)/$(upf)")
                update!(sim)
            end
            append!(averageTravelCosts, averageTravelCost(sim))
            append!(totalImprovements, totalImprovement(sim))
            append!(steps, f * upf)
        end
        df = vcat(df, DataFrame(travelCosts = averageTravelCosts, improvements=totalImprovements, steps=steps, pI=pI, pR=pR))
    end
end
CSV.write(filetext, df)

0.04 0.004 38/200 44/1000